<center>
<img src='./img/nsidc_logo.png'/>

# **IceFlow**
### Point Cloud Data Access
</center>

---
<div>
<img align="right" width="50%" height="200px" src='./img/vaex.png'/>
</div>

## Goals of this notebook
This Jupyter notebook is an **interactive document** to teach students, researchers and others interested in cryospheric sciences how to access and work with **airborne altimetry and related data sets from NASA’s [IceBridge](https://www.nasa.gov/mission_pages/icebridge/index.html) mission, and satellite altimetry data from [ICESat](https://icesat.gsfc.nasa.gov/icesat/) and [ICESat-2](https://icesat-2.gsfc.nasa.gov/) missions** using the NSIDC **IceFlow API**.

> If you want to know what an API is, take a look at this video: ["What is an API?"](https://www.youtube.com/watch?v=s7wmiS2mSXY) 

**Knowledge requirements**

To get the most out of this notebook you should be familiar with Python and its geoscience stack. If you only know some python that's also O.K. Most of the "heavy lifting" is done with our IceFlow client code so you don't necessarily need to know a lot about these libraries. If you feel like learning more about geo-science and python there are a great turorials by CU Boulder's Earth Lab here: [Data Exploration and Analysis Lessons](https://www.earthdatascience.org/tags/data-exploration-and-analysis/) or from the data carpentry project: [Introduction to Geospatial Concepts](https://datacarpentry.org/organization-geospatial/)


The main packages that we are going to use are:

 * [requests](https://requests.readthedocs.io/en/master/):
 Simple HTTP library for Python, used to make requests as its name states.
 * [geopandas](https://geopandas.org/):
 library to make working with geospatial data in python easier (using pandas). 
 * [geojson](https://github.com/jazzband/geojson):
 Functions for encoding and decoding GeoJSON formatted data in Python
 * [h5py](https://github.com/h5py/h5py):
 h5py is a thin, pythonic wrapper around the the [HDF5 library](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 
 * [matplotlib](https://matplotlib.org/):
 Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.
 * [vaex](https://github.com/vaexio/vaex):
 Vaex is a high performance Python library for lazy Out-of-Core DataFrames (similar to Pandas), to visualize and explore big tabular datasets.:
 * [iPyLeaflet](https://github.com/jupyter-widgets/ipyleaflet):
 A Jupyter / Leaflet bridge enabling interactive maps in the Jupyter notebook.
 * [icepyx](https://github.com/icesat2py/icepyx):
 a software library for ICESat-2 data users
  
As we already mentioned, for convenience we created a python library that encapsulates the most repetitive tasks to access IceFlow and Icesat 2 data. If you feel comfortable just using code you don't need to use the map widget

After completing this notebook and the companion [visualization and analysis notebook](./2_dataviz_iceflow.ipynb) you will:
* Understand the basics about the datasets served by IceFlow;
* Be able to access IceBridge, ICESat and ICESat-2 data using the IceFlow API;
* Be able to read and analyze data from IceFlow.

## **Why IceFlow**

Short answer: **data harmonization**

In 2003, NASA launched the Ice, Cloud and Land Elevation Satellite (ICESat) mission. Over the following six years, ICESat collected valuable data about ice thickness in the Polar Regions. Unfortunately, the ICESat mission ended before a follow-on mission could be launched. To fill the gap, an airborne campaign called Operation IceBridge was started. Between 2009 and 2019, Operation IceBridge flew numerous campaigns over the Greenland and Antarctic icesheets, as well as over sea ice in the Arctic and Southern Oceans. The last campaign was fill in date here. In September 2018, ICESat-2 was launched to continue NASA's collection of ice, cloud and land elevation data.

The wealth of data from these three missions, as well as from pre IceBridge airborne altimetry missions, presents an opportunity to measure the evolution of ice thickness over several decades. However, combining data from these missions presented a challenge. Data from the Airborne Topographic Mapper (ATM) flown during IceBridge campaigns is store in at least 4 different file formats. ICESat and ICESat-2 data are also in different file formats. Data needs to be harmonized (put into similar formats) before comparisons can be made. A further complication is that the coordinate reference systems used to locate measurements have changed. The Earth's surface is not static and changes shape. To account for these changes, terrestrial reference frames that relate latitude and longitude to points on the Earth are updated on a regular basis. Since the launch of ICESat, the International Terrestrial Reference Frame[(ITRF)](https://www.iers.org/IERS/EN/DataProducts/ITRF/itrf.html) has been updated three times. The geolocation accuracy of instruments means that a point measured at the beginning of the record is not the same point as that measured at the end of the record. Even though the latitude and longitude is the same. These changes in geolocation need to be reconciled if meaningful comparisons of measurements are to be made.


* A more detailed overview of these corrections can be read here: **[Applying Coordinate Transformations to Facilitate Data Comparison](https://gist.github.com/kbeamnsidc/b263eb992ce6c50a1ceafb24ac70cd0a)**



### **Pre-IceBridge**

The Airborne Topographic Mapper (ATM) is a conically-scanning laser altimeter that measures the surface topography of a swath of terrain directly beneath the path of the aircraft. ATM surveys can be used for change detection, with laser swaths re-surveyed after a few years, and differences between the two surveys yielding estimates of elevation change during the interim.  Comparison of surveys in 1993-4 and 1998-9 produced the first comprehensive assessment of the mass balance of the Greenland ice sheet (Krabill et al., 1999, 2000).  ATM surveys can also be used for comparison with satellite altimeter measurements for calibration/validation (e.g. Martin et al., 2005) or for determining elevation change.  The ATM has collected high quality topographic data from a wide variety of platforms, including the NASA P3, a Chilean Navy P3, a US Navy P3, the NASA DC8, the NCAR C-130, and a half-dozen Twin Otters. For a complete list of the ATM deployments you can go to [https://atm.wff.nasa.gov/deployments/](https://atm.wff.nasa.gov/deployments/)

### **ICESat**

ICESat (Ice, Cloud,and land Elevation Satellite) was the benchmark Earth Observing System mission for measuring ice sheet mass balance, cloud and aerosol heights, as well as land topography and vegetation characteristics. From 2003 to 2009, the ICESat mission provided multi-year elevation data needed to determine ice sheet mass balance as well as cloud property information, especially for stratospheric clouds common over polar areas. It also provided topographic and vegetation data around the globe, in addition to the polar-specific coverage over the Greenland and Antarctic ice sheets. Launched on 12 January 2003, after seven years in orbit and 18 laser-operations campaigns, the ICESat's science mission ended due to the failure of its primary instrument.


### **IceBridge**

**IceBridge** is the largest airborne survey of Earth's polar ice ever flown. It has yielded an unprecedented three-dimensional view of Arctic and Antarctic ice sheets, ice shelves and sea ice. These flights provide a yearly, multi-instrument look at the behavior of the rapidly changing features of the Greenland and Antarctic ice.
Data collected during IceBridge helps scientists bridge the gap in polar observations between NASA's Ice, Cloud and Land Elevation Satellite (ICESat) -- launched in 2003 and de-orbited in 2010 -- and ICESat-2, launched in 2018. ICESat stopped collecting science data in 2009, making IceBridge critical for extending the ice altimetry time series in the Arctic and Antarctica, although the data are not continuous.

IceBridge flights were generally conducted in **March-May over Greenland and in October-November over Antarctica**.

### **ICESat 2**

The ICESat-2 mission is designed to provide elevation data needed to determine ice sheet mass balance as well as vegetation canopy information. It will provide topographic measurements of cities, lakes and reservoirs, oceans and land surfaces around the globe. The sole instrument on ICESat-2 is the Advanced Topographic Laser Altimeter System (ATLAS), a space-based Lidar. It was designed and built at Goddard Space Flight Center, with the laser generation and detection systems provided by Fibertek. ATLAS measures the travel time of laser photons from the satellite to Earth and back; computer programs use the travel time from multiple pulses to determine elevation.

You can go to NSIDC's landing page for a complete list of IceSat 2 datasets and their documentation: [ICESat-2 Data Sets at NSIDC](https://nsidc.org/data/icesat-2/data-sets)

<p align="center">
<img style="align: center;" width="80%" src='./img/iceflow-coverage.jpg'/>
    <br>
    <b>Fig 2. IceFlow mission coverages</b>
</p>


## Data sets and their coverage


The IceFlow project provides web services for ordering spatially and temporally subseted Lidar point cloud data from the [BLATM L1B](https://nsidc.org/data/BLATM1B), [ILATM L1B v1](https://nsidc.org/data/ilatm1b/versions/1), [ILATM L1B V2](https://nsidc.org/data/ILATM1B), [ILVIS2](https://nsidc.org/data/ILVIS2) and [IceSat GLAH06](https://nsidc.org/data/GLAH06/) data products.

The following table describes the temporal and spatial coverage of each of these dataset as well as the sensor and platform used to acquire the data.


---

|              | Spatial Coverage                                                      | Temporal Coverage                              | Platform                                              | Sensor                   |
|--------------|-----------------------------------------------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------|
| [BLATM L1B](https://nsidc.org/data/BLATM1B)    | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 23 June 1993 - 30 October 2008                 | DC-8, DHC-6, P-3A ORION, P-3B                         | ATM                      |
| [ILATM L1B v1](https://nsidc.org/data/ilatm1b/versions/1) | South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 31 March 2009 - 8 November 2012 (updated 2013) | AIRCRAFT, DC-8, P-3B                                  | ATM                      |
| [ILATM L1B V2](https://nsidc.org/data/ILATM1B)| South: N:-53, S: -90, E:180, W:-180 North: N:90, S: 60, E:180, W:-180 | 20 March 2013 - 16 May 2019 (updated 2020)     | C-130, DC-8, HU-25A, HU-25C, P-3B, WP-3D ORION        | ATM                      |
| [ILVIS2](https://nsidc.org/data/ILVIS2)       | North: N:90, S: 60, E:180, W:-180                                     | 25 August 2017 - 20 September 2017             | AIRCRAFT, B-200, C-130, DC-8, G-V, HU-25C, P-3B, RQ-4 | ALTIMETERS, LASERS, LVIS |
| [GLAH06](https://nsidc.org/data/GLAH06/)       | Global: N:86, S: -86, E:180, W:-180                                     |     20 February 2003 - 11 October 2009        | IceSat | ALTIMETERS, CD, GLAS, GPS, GPS Receiver, LA, PC


--- 

> **Note**: If you have any qustions about the data please contact NSIDC user services at nsidc@nsidc.org

In this tutorial we are going to use iPyLeaflet and other Jupyter widgets to select our constraints and place a data order using the IceFlow API.

### NASA's EarthData Credentials

The first step to start working with IceFlow data is to login into [NASA's Earthdata Search](https://earthdata.nasa.gov/).

> **Note**: you don't need to be a NASA employee to register with NASA EarthData!


In [ ]:
# We import our IceFlow client library
from iceflow.client import IceflowClient
# We instantiate our client
iceflow = IceflowClient()
# We need to use our NASA Earth Data Credentials and verify that they work.
# Please click on set credentials and then see if we are authenticated by executing the next cell.
iceflow.display(['credentials'])

In [ ]:
# We are going to verify that our credentials are valid.
session = iceflow.create_earthdata_authenticated_session()
if session is None:
    print('we are not logged into NASA EarthData')
else:
    print('we are logged into NASA EarthData!')

In [ ]:
# Let's start with the user interface. We'll explain what this does next.
# vertical = Sidecar widget, horizontal = render the widget in this notebook.
iceflow.display(['map'], 'vertical', extra_layers=True)

This user interface uses [ipylaflet](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a) which allows us to draw
polygons or bounding boxes to delimit our area of interest. We can also edit and delete these geometries using the the widget controls in the map.

The **"Get Granule Count"** button will query [NASA's CMR](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr) to get a granule count for the current parameters, we need to have a geometry and one or more datasets selected.

**Notes**: 
> * If you use the bounding box geometry in a polar projection you'll notice a distortion due the nature of polar coordinates, if you prefer you can use the global mercator map to draw a bounding box without apparent distortion. Polygons are probably a better idea or you can even input your own coordinates as we'll see later.
> * The calculated download size of these granules is an upper bound since IceFlow allows us to subset the data. 

## IceFLow notebook interface components

* **Hemisphere**: Which map projection we are going to use, we can pick global, north or south
* **Datasets**: A selection of the datasets served by IceFlow, we can pick one or more (CTRL+ Space or CTRL+Click) In the widget ATM includes the 3 different ATM products (BLATM L1B, ILATM L1B v1, ILATM L1B V2)
* **IceSat2**: If we want to also place a data order for IceSat 2 data for the current parameters we need to select the short name code i.e. ATL06
* **ITRF**: The International Terrestrial Reference Frame see: [ITRF](https://gist.github.com/kbeamnsidc/b263eb992ce6c50a1ceafb24ac70cd0a)
* **Epoch**: The epoch in which the coordinate reference systems are based, valid when using ITRF.
* **Date Range**: A slider control to filter the selection area between a start and end date
* **Map**: The main widget, you can draw polygon or bounding boxes and edit them. You can also turn on and off the layers that show IceBridge flights and the rest.

To display the user interface we use the method `display` and we need 3 parameters:

* **what**: what we want to display, the valid values are **credentials**, **controls** and **map**
* **where**: where we want the widgets to be, horizontal will render them in the next cell or vertical will use a separate column.
* **extra_layers**: if we want to add more than the IceBridge layers to the map, True or False

> **Note:** The IceFlow client can work directly with the data ordering system. If you are logged into NASA you can just build your own parameters and send them directly without using the user interface.

## NASA Common Metadata Repository (CMR)

What is CMR?

NASA's Common Metadata Repository (CMR) is a high-performance, high-quality, continuously evolving metadata system that catalogs all data and service metadata records for NASA's Earth Observing System Data and Information System (EOSDIS) system and will be the authoritative management system for all EOSDIS metadata. These metadata records are registered, modified, discovered, and accessed through programmatic interfaces leveraging standard protocols and APIs. 

**In short: NASA's CMR is a "database" for Earth-related datasets, in this case all the data served by IceFlow is also indexed by CMR**

> **Note**: One **important** thing to notice here is that CMR has the location of the original data granules and thus they are in multiple data formats and projections. Again, here is where tools like IceFlow come handy. Also, the data size calculation for CMR granules is an upper bound since CMR does not subsets the data.

In [ ]:
# Use the widget statate to build spatio-temporal parameters
params = iceflow.build_params()
params

In [ ]:
# We can query CMR to get an idea of coverages for the area we just selected.
# The granules and total size is an upper bound since CMR has full granules
# and IceFlow will subset them to only cover the area selected.
# datasets = ['BLATM1B', 'GLAH06', 'ILVIS2', 'ATL06']
datasets = ['BLATM1B', 'GLAH06', 'ILVIS2']
granules = iceflow.query_cmr(datasets, params)

In [ ]:
## We can print the first record for a given dataset
dataset = 'BLATM1B'
if len(granules[dataset])>0:
    print(granules[dataset][0])

## IceFlow API

The native IceFlow API offers us an Open API specification that we can use to explore how the service works. 

IceFlow has 3 dataset end-points and endpoints to check the status of previous data orders. Each data set endpoint allows users to specify their spatial and temporal extent. 

**Spatial Extent Parameters**

* **Polygon**: A conterclockwise closed array of lat-lons, the last coordinate has to be the same as the first pair.
* **bounding box**: a WGS84 box with min_lon,min_lat,max_lon,max_lat values

**Temporal Extent Parameters**
* **date_range**: The date/time range over which to return data, accepts UTF datetime or simple YYYY-mm-dd formatted values.

**ITRF** (optional)
* The ITRF reference to which the data will be transformed via the published ITRF transformation parameters. Optional, but must be used when specifying epoch.
* Available values : **ITRF2000, ITRF2008, ITRF2014**

**Epoch** (optional)
* The epoch (in decimal years) to which the data will be transformed via the ITRF Plate Motion Model corresponding to ITRF. Optional, and can be used when also specifying itrf, but can only be used if itrf is ITRF2008 or ITRF2014 since there is no ITRF2000 Plate Motion Model.


In [ ]:
from IPython.display import IFrame
IFrame('http://valkyrie-vm.apps.nsidc.org/1.0/ui/#/', width=800, height=400)

## Placing a data order for one of the data sets served by IceFlow

Now that we have our constraints we just need to post our order and wait for IceFlow to fulfill it. 

We can put an order directly, in this case we are going to work on a geometry that overlaps with [Jakobshavn](https://en.wikipedia.org/wiki/Jakobshavn_Glacier) glacier in Greenland or Thwaites Glacier in Antarctica (depending on which one you select). 




In [ ]:
# note that we are explicitly using the name of one of the ATM datasets
# dataset = 'ILATM1B'
dataset = 'GLAH06'

# Thwaites glacier
my_params ={
    'dataset': 'GLAH06',
    'start': '2001-01-01',
    'end': '2011-1-01',
    'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'
}

# Jakobshavn 2016
# my_params ={
#     'dataset': dataset,
#     'start': '2016-01-01',
#     'end': '2016-12-31',
#     'bbox': '-107.4515,-75.3695,-105.3794,-74.4563'
# }

# returns a json dictionary, the request parameters and the order's response.
order = iceflow.post_iceflow_order(my_params)
order

## Downloading the data
Let's get some coffee, some IceFlow orders are in the Gigabytes real and may take a little while to be processed. 
Once that your status URL says is completed we can grab the HDF5 data file using the URL on the same response!

The first thing we need to know is the order status

In [ ]:
order['response'].json()

In [ ]:
order_status = iceflow.order_status(order)
if 'url' in order_status:
    data_url = order_status['url']
order_status

## Data order is complete
Now that we know the order is complete we can download the file in our status response `url` to our local working directory using requests or curl or wget or just clicking the link.



In [ ]:
# We can name our file or leave it with the original file name by not using the file_name parameter.
file_name = 'twaties-test-GLAH06-2000-2010'
iceflow_file = iceflow.download_order(data_url, file_name)

## Placing multiple data orders to IceFlow

If we need more than one data set from IceFlow at the same time, we can use the client to place our data orders, IceFlow will return a single HDF5 file for each of the data sets that we have selected using the map widget.

In [ ]:
# orders = v.post_orders()
# orders

## Downloading Related Datasets

* **TODO**: [IcePyx](https://github.com/icesat2py/icepyx/blob/development/examples/ICESat-2_DAAC_DataAccess2_Subsetting.ipynb) + IceFlow Downloader example: 


## Reading the Data

Remote sensing data can be overwhelmingly big. Reading a big file is not trivial and when we have an array of them this task can become an intractable barrier.
The main constraint if you don't have a super computer is memory. The average granule size is in the 10s of MB for IceSat 2 and could be Gigabytes in IceFlow depending on the selected area. This is when libraries like Dask, Vaex and others come into play. 

These libraries read our files using a battery of optimizations like lazy loading, memory mapping and parallelism. Let's now explore 4 different ways of reading these HDF5 files using libraries included in this notebook:

* h5py + Pandas
* Dask arrays
* Vaex
* xarray

## What's in the IceFlow HDF5 file?

**TODO**: write about the important fields on the datasets?

## h5py + pandas

With h5py we get almost native access to the hdf5 files and we can use pandas or geopandas to compute operations on them.



In [ ]:
# importing all of our dependencies
import warnings
warnings.filterwarnings("ignore")
import glob
import geopandas
import pandas as pd
import h5py
import vaex
import dask.dataframe as dd
import dask.array as da
import numpy as np

In [ ]:
%%time

# file_name = 'data/atm1b_data_2020-07-11T20-39.hdf5'
file_name = 'data/twaties-test-GLAH06-2000-2010.h5'

f = h5py.File(file_name, 'r')
# We are going to print our variables
print(list(f.keys()))

glas_data = {
    'latitude': f['d_lat'],
    'longitude': f['d_lon'],
    'elevation': f['d_elev'],
    'time': pd.to_datetime(f['utc_datetime'])
}

# Dictionary for ATM data
# df_data = {
#     'latitude': f['latitude'],
#     'longitude': f['longitude'],
#     'elevation': f['elevation'],
#     'time': pd.to_datetime(f['utc_datetime'])
# }

# Dictionary for ATM data
df = pd.DataFrame(data=glas_data)
display(df)
# compute the mean elevation subsetting by coordinates
df[ df.latitude < -74.939994 ].elevation.mean()

# Xarray

In [ ]:
%%time
import xarray as xr
fname = 'data/atm1b_data_2020-07-11T20-39.hdf5'
import xarray as xr
ds = xr.open_dataset(fname) 
ds

# Vaex

In [ ]:
%%time
import vaex
df = vaex.open('data/atm1b_data_2020-07-11T20-39.hdf5')
# We're parsing the utc_datetime from IceFlow into a data type that Vaex understands.
df['date'] = df.utc_datetime.values.astype('datetime64[ns]')
my_df = df['longitude', 'latitude', 'elevation', 'date']
my_df.describe()

### References
1. [Airborne Topographic Mapper Calibration Procedures and Accuracy Assessment](https://ntrs.nasa.gov/archive/nasa/casi.ntrs.nasa.gov/20120008479.pdf)

2. [Open Source Tools for Point Cloud Processing, Storage, Subsetting, and Visualization](https://sea.ucar.edu/sites/default/files/kbeam_seaconf18.pdf)

### Related Tools

* [OpenAltimetry](https://openaltimetry.org/): Advanced discovery, processing, and visualization services for ICESat and ICESat-2 altimeter data
* [ITS_LIVE](https://its-live.jpl.nasa.gov/):A NASA MEaSUREs project to provide automated, low latency, global glacier flow and elevation change datasets. 